In [1]:
!pip install torch torchvision matplotlib


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Subset
import numpy as np

# Define transforms (basic normalization, no heavy augmentations)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-100 dataset
full_trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
full_testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

# Reduce dataset to 10,000 training samples (out of 50,000)
np.random.seed(42)  # For reproducibility
subset_indices = np.random.choice(len(full_trainset), 10000, replace=False)  
trainset = Subset(full_trainset, subset_indices)

# Reduce test set to 2,000 samples (from 10,000)
test_indices = np.random.choice(len(full_testset), 2000, replace=False)  
testset = Subset(full_testset, test_indices)

# Create DataLoaders (smaller batch size for CPU)
batch_size = 64
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)


100%|██████████| 169M/169M [00:23<00:00, 7.08MB/s] 


In [3]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

# Load ResNet models (modify output layer for 100 classes)
def get_model(model_name):
    if model_name == "resnet18":
        model = models.resnet18(weights="IMAGENET1K_V1")
    elif model_name == "resnet34":
        model = models.resnet34(weights="IMAGENET1K_V1")
    
    model.fc = nn.Linear(model.fc.in_features, 100)  # Change output layer for CIFAR-100
    return model.to("cpu")  # Ensure model runs on CPU

model_18 = get_model("resnet18")
model_34 = get_model("resnet34")


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\anand/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 25.2MB/s]
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to C:\Users\anand/.cache\torch\hub\checkpoints\resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:03<00:00, 23.8MB/s]


In [4]:
import time

def train_model(model, trainloader, criterion, optimizer, num_epochs=5):
    model.train()
    
    for epoch in range(num_epochs):
        start_time = time.time()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        epoch_time = time.time() - start_time
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}, "
              f"Accuracy: {100 * correct/total:.2f}%, Time: {epoch_time:.2f}s")

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()

optimizer_18 = optim.Adam(model_18.parameters(), lr=0.001)
optimizer_34 = optim.Adam(model_34.parameters(), lr=0.001)

# Train ResNet-18
print("Training ResNet-18...")
train_model(model_18, trainloader, criterion, optimizer_18, num_epochs=5)

# Train ResNet-34
print("\nTraining ResNet-34...")
train_model(model_34, trainloader, criterion, optimizer_34, num_epochs=5)


Training ResNet-18...
Epoch 1, Loss: 3.6779, Accuracy: 15.60%, Time: 55.78s
Epoch 2, Loss: 2.7921, Accuracy: 28.45%, Time: 54.77s
Epoch 3, Loss: 2.3021, Accuracy: 38.60%, Time: 55.12s
Epoch 4, Loss: 1.9155, Accuracy: 47.88%, Time: 54.83s
Epoch 5, Loss: 1.5856, Accuracy: 55.08%, Time: 55.38s

Training ResNet-34...
Epoch 1, Loss: 3.9051, Accuracy: 11.35%, Time: 96.00s
Epoch 2, Loss: 2.9520, Accuracy: 25.51%, Time: 95.66s
Epoch 3, Loss: 2.3940, Accuracy: 37.08%, Time: 95.58s
Epoch 4, Loss: 1.9888, Accuracy: 46.27%, Time: 95.71s
Epoch 5, Loss: 1.6698, Accuracy: 53.31%, Time: 95.69s


In [5]:
def evaluate_model(model, testloader):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0
    start_time = time.time()

    with torch.no_grad():
        for inputs, labels in testloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    inference_time = time.time() - start_time
    accuracy = 100 * correct / total
    avg_loss = test_loss / len(testloader)
    
    return accuracy, avg_loss, inference_time

# Evaluate ResNet-18
acc_18, loss_18, time_18 = evaluate_model(model_18, testloader)
print(f"ResNet-18 - Accuracy: {acc_18:.2f}%, Loss: {loss_18:.4f}, Inference Time: {time_18:.2f}s")

# Evaluate ResNet-34
acc_34, loss_34, time_34 = evaluate_model(model_34, testloader)
print(f"ResNet-34 - Accuracy: {acc_34:.2f}%, Loss: {loss_34:.4f}, Inference Time: {time_34:.2f}s")


ResNet-18 - Accuracy: 34.15%, Loss: 2.8005, Inference Time: 7.74s
ResNet-34 - Accuracy: 37.35%, Loss: 2.8934, Inference Time: 8.27s
